In [1]:
import random
import numpy as np

**Perceptron Class**

Here we create a class for the Perceptron that we may use to instatiate any number of Perceptron instances.

In [2]:
class Perceptron:
    
    def __init__(self, num_inputs, act='sigmoid'):
        self.weights = []
        self.num_inputs = num_inputs
        self.act = act # define activation function with sigmoid being the default
        for _ in range(0, num_inputs):
            self.weights.append(random.random() * 2 - 1)
        print(self.weights)
            
    def get_weights(self):
        return self.weights
        
    def feed_forward(self, inputs):
        self.inputs = inputs
        sum = 0
        
        # multiply inputs by weights and sum them
        for i in range(0, self.num_inputs):
            sum += self.weights[i] * inputs[i]
            
        # 'activate' the sum and get the derivative
        self.output, self.output_prime = self.activate(sum)
        return self.output
    
    def activate(self, x):
        if (self.act == 'sigmoid'):
            activation = self.sigmoid(x)
            activation_prime = activation * (1 - activation)
        else:
            activation = self.step(x)
            activation_prime = 1 # use 1 since step activation is not differentiable
        return activation, activation_prime
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def step(self, x):
        if x > 0:
            return 1
        return 0
    
    def backward_pass(self, error):
        learning_rate = 0.01 # hyperparameter
        back_error = [] # each element in list represent amount of error to send backward along that connection
        for i in range(0, self.num_inputs):
            back_error.append(error * self.output_prime * self.weights[i])
            self.weights[i] -= error * self.output_prime * self.inputs[i] * learning_rate
        return back_error

**Single Perceptron to classify a point as being above or below a line**

Begin with a line going through the origin.

In [44]:
def line(x):
    return 0.5 * x

In [56]:
p = Perceptron(2, act='step')
weights = p.get_weights()
print(sum(weights))

[-0.08058198345153089, 0.9651201836932843]
0.8845382002417534


In [79]:
# pick 1 random point and determine if it's above or below the line
x_coord = random.random() * 1000
y_coord = random.random() * 1000
line_y = line(x_coord)

print(x_coord, y_coord)
print(x_coord, line_y)

if y_coord > line_y:
    answer = 1
else:
    answer = 0
    
print(answer)

901.9672274233975 255.103627952859
901.9672274233975 450.98361371169875
0


In [80]:
# pass the random point into the node, feed it back and see if the weights change
guess = p.feed_forward([x_coord, y_coord])
p.backward_pass(guess - answer)
p.get_weights()

[-8.685332254545209, -2.38793593594849]


[-8.685332254545209, -2.38793593594849]

In [57]:
# train the network
for _ in range(0, 1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-18.29384246820456, 36.81655593994777]

In [58]:
# check the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    is_above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (is_above == True and guess_above >= 0.5):
        correct += 1
    if (is_above == False and guess_above < 0.5):
        correct += 1

print(correct)

999


Add bias to the node in order to handle lines that do not go through the origin.

In [71]:
def line(x):
    return 0.5 * x + 500

In [78]:
p = Perceptron(3, act='step')
weights = p.get_weights()

[0.2214871748797087, 0.17398763304982867, -0.5255616218064509]


In [79]:
# train the Perceptron using a bias term
for _ in range(0, 100000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord, 1])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-15.181007899220747, 13.873452267396452, -65.51556162180306]

In [80]:
# determine the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (above == True and guess_above >= 0.5):
        correct += 1
    if (above == False and guess_above < 0.5):
        correct += 1

print(correct)

800


**Perceptrons for Logic**

Implement logicial AND and OR

In [58]:
a = Perceptron(3) # logical AND
b = Perceptron(3) # logical OR

[0.1973171969546439, 0.9407410789339032, -0.8785536451209432]
[0.8440494165572345, -0.7232351403105719, 0.1247101699302735]


In [65]:
# Train the 'a' Perceptron for logical AND
for _ in range(0,100000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    a_out = a.feed_forward([first, second, 1])
    
    if (first and second):
        answer = 1
    else:
        answer = 0
    a.backward_pass(a_out - answer)
    
print(a.get_weights())

# Train the 'b' Perceptron for logical OR
for _ in range(0,100000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    b_out = b.feed_forward([first, second, 1])
    
    if (first or second):
        answer = 1
    else:
        answer = 0
    b.backward_pass(b_out - answer)
    
print(b.get_weights())

[5.153930989203262, 5.152757919753352, -7.824237161307208]
[5.854616254553259, 5.851694138766977, -2.675989289602662]


In [66]:
print("Logical AND")
print(a.feed_forward([1, 1, 1]))
print(a.feed_forward([1, 0, 1]))
print(a.feed_forward([0, 1, 1]))
print(a.feed_forward([0, 0, 1]))
print("\nLogical OR")
print(b.feed_forward([1, 1, 1]))
print(b.feed_forward([1, 0, 1]))
print(b.feed_forward([0, 1, 1]))
print(b.feed_forward([0, 0, 1]))

Logical AND
0.922902429355317
0.06474842555742528
0.06467742534473865
0.00039976367222106004

Logical OR
0.9998802902889667
0.9600220025673405
0.9599097015938103
0.06440512841264989


**XOR with 2 Perceptrons**

In [40]:
a = Perceptron(3, act='sigmoid')
b = Perceptron(4, act='sigmoid')

[-0.7324361935978867, -0.4001735717141355, 0.08008286093765427]
[-0.32654986517018636, -0.46175675370799096, 0.08301394583566357, 0.5574669963236427]


In [32]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, a_out, second, 1])
    return b_out

In [33]:
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, a_out, second, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = b.backward_pass(b_out - answer)
    a.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())

[-7.483987979457412, -7.483897410865939, 2.84301037230098]
[-5.827465050510558, -12.897213765778515, -5.829549347096561, 8.861141901288946]


In [34]:
print(network(1, 1))
print(network(1, 0))
print(network(0, 1))
print(network(0, 0))

0.0575438332206778
0.9483596411558143
0.9482569276656566
0.034684539885014534


**XOR Network with 3 Perceptrons**

In [35]:
a = Perceptron(3, act='step')
b = Perceptron(3, act='step')
c = Perceptron(3, act='step')

[-0.026414675884606442, -0.5541296592695797, -0.732296699393026]
[0.8744238644234399, -0.8816360334074824, 0.5238647800943286]
[0.09005270892545592, 0.39116154747133836, -0.36227963612987457]


In [36]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    return c_out

In [38]:
# train the network
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    c_out = network(first, second)
#     a_out = a.feed_forward([first, second, 1])
#     b_out = b.feed_forward([first, second, 1])
#     c_out = c.feed_forward([a_out, b_out, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())
print(c.get_weights())

[0.6994101580545675, -1.2259228678535024, -0.6989771970906062]
[0.7109859370424579, -0.8165011775436793, 0.42749455265870145]
[0.09005270892545592, -0.008838452528661856, 0.007720363870125613]


In [39]:
print(network(1, 1))
print(network(1, 0))
print(network(0, 1))
print(network(0, 0))

0
1
1
0


In [131]:
print('%f' % a.feed_forward([1, 1, 1]))
print('%f' % a.feed_forward([1, 0, 1]))
print('%f' % a.feed_forward([0, 1, 1]))
print('%f' % a.feed_forward([0, 0, 1]))
print("\n")
print('%f' % b.feed_forward([1, 1, 1]))
print('%f' % b.feed_forward([1, 0, 1]))
print('%f' % b.feed_forward([0, 1, 1]))
print('%f' % b.feed_forward([0, 0, 1]))
print("\n")
print('%f' % c.feed_forward([1, 1, 1]))
print('%f' % c.feed_forward([1, 0, 1]))
print('%f' % c.feed_forward([0, 1, 1]))
print('%f' % c.feed_forward([0, 0, 1]))

1.000000
0.000000
0.000000
0.000000


1.000000
1.000000
1.000000
0.000000


0.000000
0.000000
1.000000
0.000000


**Parabola**

Utilizing non-lenear activation

In [3]:
def parabola(x):
    return 0.005 * pow(x - 500, 2) + 250

In [58]:
# a = Perceptron(3, act='step')
# b = Perceptron(3, act='step')
# c = Perceptron(3, act='step')
a = Perceptron(3, act='sigmoid')
b = Perceptron(3, act='sigmoid')
c = Perceptron(3, act='sigmoid')

def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    return c_out

[0.20917815737193202, -0.23639400057079452, 0.27495596475726036]
[-0.5095881229167303, -0.9707882568372936, -0.6790186068221049]
[-0.7317177330177915, -0.1529564810869779, -0.1411097134146675]


In [59]:
for _ in range(0,2000000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    curve_y = parabola(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000

    c_out = network(x_norm, y_norm)
    
    if y_coord > curve_y:
        answer = 1
    else:
        answer = 0

    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
nodea = a.get_weights()
print(nodea)
print(-nodea[0]/nodea[1], -nodea[2]/nodea[1])
nodeb = b.get_weights()
print(nodeb)
print(-nodeb[0]/nodeb[1], -nodeb[2]/nodeb[1])
print(c.get_weights())

[12.190508786880516, -5.056695774321589, -5.818481793465892]
2.4107657116303396 -1.1506489718073865
[-12.39864219343587, -5.090676723126714, 6.42003725408214]
-2.435558741553829 1.2611363092290266
[-13.959946650288039, -14.187156634027685, 8.001571238496876]


In [60]:
# determine the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    curve_y = parabola(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000
    
    is_above = y_coord > curve_y
    guess_above = network(x_norm, y_norm)
    
    if (is_above == True and guess_above >= 0.5):
        correct += 1
    if (is_above == False and guess_above < 0.5):
        correct += 1

print(correct)

988


In [61]:
print('%f' % c.feed_forward([1, 1, 1]))
print('%f' % c.feed_forward([1, 0, 1]))
print('%f' % c.feed_forward([0, 1, 1]))
print('%f' % c.feed_forward([0, 0, 1]))

0.000000
0.002577
0.002055
0.999665


In [62]:
def line_a(x):
    return 2.4818372680469687 * x - 1146.7840987207953
def line_b(x):
    return -2.5104326279010922 * x + 1313.663879791022

In [63]:
# check the accuracy
correct_a = 0
correct_b = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y_a = line_a(x_coord)
    line_y_b = line_b(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000
#     print(x_coord, y_coord, line_y_a, line_y_b)
    
    is_above_a = y_coord > line_y_a
    is_above_b = y_coord > line_y_b
    guess_above_a = a.feed_forward([x_norm, y_norm, 1])
    guess_above_b = b.feed_forward([x_norm, y_norm, 1])
#     print(is_above_a, is_above_b, guess_above_a, guess_above_b)
    
    if (is_above_a == True and guess_above_a >= 0.5):
        correct_a += 1
    if (is_above_a == False and guess_above_a < 0.5):
        correct_a += 1
    if (is_above_b == True and guess_above_b >= 0.5):
        correct_b += 1
    if (is_above_b == False and guess_above_b < 0.5):
        correct_b += 1

print(correct_a)
print(correct_b)

26
13
